# Get COVID-Related pages using pagelinks Dumps and  wmf.wikidata_item_page_link

**All the data used in this code is public**. In this notebook we get the  COVID-19 related pages, using the pagelink SQL WMF dumps. For efficiency, we use the implementation wikidata_item_page_link based on the [WMF's Hadoop Cluster](https://wikitech.wikimedia.org/wiki/Analytics/Systems/Cluster), but the experiments are possible to reproduce changing that table for the [Wikidata SQL dumps](https://dumps.wikimedia.org/wikidata)

## Preprocess

* First run a zgrep on  [wikidata pagelinks sql dump](http://dumps.wikimedia.org/wikidatawiki/latest/wikidatawiki-latest-pagelinks.sql.gz), prefiltering lines containing links to  the 3 seeds (Q81068910,Q82069695,Q84263196).
         zgrep "Q81068910\|Q82069695\|Q84263196" wikidatawiki-latest-pagelinks.sql.gz > Q81068910_Q82069695_Q84263196_zgrepmatch.txt
* Put resultsin Q81068910_Q82069695_Q84263196_zgrepmatch.txt


!zgrep "Q81068910\|Q82069695\|Q84263196" wikidatawiki-latest-pagelinks.sql.gz > Q81068910_Q82069695_Q84263196_zgrepmatch.txt

In [1]:
#Open the zgrep output
f = open('Q81068910_Q82069695_Q84263196_zgrepmatch.txt') 

FileNotFoundError: [Errno 2] No such file or directory: 'Q81068910_Q82069695_Q84263196_zgrepmatch.txt'

In [50]:
#Here I refine the zgrep, that contains many tuples per line
#The output here is the page id of the wikidata item, for example:
#page_id -> 23075, wikidata_item -> Q189067 , wikidata_label ->Alyssa_Milano
import re
wikidataPagesIds = []
seeds = ['Q81068910','Q82069695','Q84263196']
for l  in f:
    for s in seeds:
        wikidataPagesIds.extend(re.findall("\(([0-9]+),0,\'%s\'" % s,l))

In [ ]:
# From the ids get wikidata_item

# This query could be replaced or build from the 'page' sql table, on this file:
# https://dumps.wikimedia.org/wikidatawiki/latest/wikidatawiki-latest-page.sql.gz

wikidataPageTitles = spark.sql('''SELECT page_id,page_title FROM wmf.mediawiki_history WHERE snapshot = "2020-08"
                                AND wiki_db="wikidatawiki"''')
wikidataPageTitles = wikidataPageIds.where(wikidataPageIds.page_id.isin(wikidataPagesIds))
#get just the Q values (wikidata ids)
QsList = wikidataPageTitles.select('page_title').collect()
#Readapt format
QsList = [x.page_title for x in QsList]

In [76]:
# Our subset of languages
dbs = ['fr', 'zh', 'is', 'ar', 'de', 'en', 'ru', 'pt', 'ko', 'it', 'ja', 'es']
wiki_dbs = [db+'wiki' for db in dbs]

In [56]:
# This table could be replace or build from:
# https://dumps.wikimedia.org/wikidatawiki/latest/wikidatawiki-latest-wb_items_per_site.sql.gz

pageId = spark.sql('''SELECT 
item_id as wikidata_item, page_id, page_title,wiki_db FROM wmf.wikidata_item_page_link  
WHERE snapshot="2020-08-24" AND page_namespace=0  
''')

In [62]:
pagesWithQs = pageId.where(pageId.wikidata_item.isin(QsList))


In [66]:
pagesWithQs.show()

+-------------+--------+--------------------+----------+
|wikidata_item| page_id|          page_title|   wiki_db|
+-------------+--------+--------------------+----------+
|    Q66433557|61908507|         Luca_Kilian|    enwiki|
|    Q10304982|    8215|      Jair_Bolsonaro|    rwwiki|
|      Q189067|   23075|       Alyssa_Milano|    cswiki|
|      Q271933|  564021|         패트릭_유잉|    kowiki|
|     Q5362050|  751054|Eliyahu_Bakshi-Doron|simplewiki|
|    Q84055544|  694572|Filippinda_COVID-...|    uzwiki|
|      Q429193| 2600617|     Maxime_Gonalons|    ptwiki|
|      Q180589|  559017|       Boris_Johnson|    eowiki|
|    Q55030082|  972444|     Շառլոթ_Լոուրենս|    hywiki|
|       Q82805|  948774|        Sergio_Pérez|    nowiki|
|    Q89598747|  693873|San-Tome_va_Prins...|    uzwiki|
|    Q11720946| 2465945|       Jarosław_Mika|    plwiki|
|     Q3350076|   87392|      ਮਸ਼ਰਫ਼_ਮੋਰਤਜ਼ਾ|    pawiki|
|     Q6110020| 5024910|       Jabari_Parker|    itwiki|
|     Q4717362| 7760103|            

In [77]:
pagesWithQsAndWikis = pagesWithQs.where(pageId.wiki_db.isin(wiki_dbs))

In [78]:
pagesWithQsAndWikisPd = pagesWithQsAndWikis.toPandas()

In [80]:
pagesWithQsAndWikisPd.wiki_db.value_counts()

enwiki    1920
eswiki     990
frwiki     914
dewiki     897
arwiki     846
ptwiki     690
itwiki     660
zhwiki     626
ruwiki     503
kowiki     464
jawiki     389
iswiki      28
Name: wiki_db, dtype: int64

In [101]:
#Here I read wooks data to get the Qs labels

import glob, pandas as pd
languagesWook = glob.glob('wikp*json')
pages = {}
for lang in languagesWook:
    langcode =lang[-7:-5]
    pages[langcode] = pd.read_json(lang)
    pages[langcode]['db'] = '%swiki' % langcode
    pages[langcode]['project'] = langcode
allpages = pd.concat(pages.values())
QsWook = allpages[['QID','category']].drop_duplicates()

dataWithLabels= pagesWithQsAndWikisPd.set_index('wikidata_item').join(QsWook.set_index('QID'))
print(dataWithLabels.shape, pagesWithQsAndWikisPd.shape)

(8927, 4) (8927, 4)


In [102]:
for db, data in dataWithLabels.groupby('wiki_db'):
    data.to_csv('wikipedia_list_clean_data_v2_%s.csv' % db,index=False)

#### Wook data cleaned by me


* enwiki    1715
* eswiki     917
* frwiki     833
* dewiki     817
* arwiki     757
* ptwiki     638
* itwiki     599
* zhwiki     576
* ruwiki     458
* kowiki     430
* jawiki     338
* iswiki      23

# Computing coverage by language (For all languages)

In [84]:
# Coverage by languages

differentQs = pagesWithQs.select('wikidata_item').distinct().count()

In [85]:
#Here I get the pages per wiki, but for all wikis not just the one in our subset of wikis
pagesPerWiki = pagesWithQs.groupBy('wiki_db').count().toPandas()

In [86]:
differentQs

2233

In [ ]:
pagesPerWiki['coverage_ratio'] = pagesPerWiki['count'] /differentQs

In [93]:
pagesPerWiki = pagesPerWiki[pagesPerWiki.wiki_db.str.endswith('wiki')] #removing wikinews wicktionary, etc

In [95]:
pagesPerWiki.sort_values('coverage_ratio',ascending=False)

,wiki_db,count,coverage_ratio
48,enwiki,1920,0.859830
193,eswiki,990,0.443350
235,trwiki,971,0.434841
173,frwiki,914,0.409315
190,dewiki,897,0.401702
194,arwiki,846,0.378863
209,simplewiki,725,0.324675
158,ptwiki,690,0.309001
118,itwiki,660,0.295567
174,zhwiki,626,0.280340


In [96]:
pagesPerWiki.sort_values('coverage_ratio',ascending=False).to_csv('coverage_of_Covid_pages_per_wiki.csv',index=False)